In [7]:
!pip3 install weaviate-client

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.6/95.6 kB 988.2 kB/s eta 0:00:00a 0:00:01
  Using cached requests-2.28.2-py3-none-any.whl (62 kB)
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.1/77.1 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 214.8/214.8 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 51.6 MB/s eta 0:00:00a 0:00:01
  Using cached charset_normalizer-3.1.0-cp310-cp310-macosx_11_0_arm64.whl (123 kB)
  Using cached urllib3-1.26.15-py2.py3-none-any.whl (140 kB)
  Using cached certifi-2023.5.7-py3-none-any.whl (156 kB)
  Using cached idna-3.4-py3-none-any.whl (61 kB)
  Using cached cffi-1.15.1-cp310-cp310-macosx_11_0_arm64.whl (174 kB)
  Using cached pycparser-2.21-py2.py3-none-any.whl (118 kB)
  Created wheel for validators: filename=validators-0.20.0-py3-none-any.whl size=19581 sha256=15071ffb5d5aa00174a0ec6111036180542f49144880af27b2d4cfd158f164a5
  Stored in directory: /

In [8]:
import weaviate
import json
client = weaviate.Client(
    url = "http://localhost:8080/",  # Replace with your endpoint
    additional_headers = {
        "X-OpenAI-Api-Key": ""  # Replace with your API key
    }
)

In [18]:
# ===== import data =====
# Load local JSON file
with open('flows.json') as f:
    flows = json.load(f)

# Prepare a batch process
with client.batch as batch:
    batch.batch_size=100
    # Batch import all Questions
    for i, d in enumerate(flows):
        # print(f"importing question: {i+1}")  # To see imports

        properties = {
            "description": d["description"],
            "flow": d["flow"],
            "source": d["source"],
            "source_id": d["source_id"],
            "url": d["url"],
            "created_at": d["created_at"],
            "author": d["author"],
        }

        client.batch.add_data_object(properties, "Flow")
client.batch.flush()

To get all flows

In [19]:
all_objects = client.data_object.get(class_name="Flow")
print(all_objects)

{'deprecations': [], 'objects': [{'class': 'Flow', 'creationTimeUnix': 1683755281008, 'id': '071605ec-6d7d-49d5-ae80-c9e8efb83da9', 'lastUpdateTimeUnix': 1683755281008, 'properties': {'author': 'Alice', 'created_at': '2021-01-01T12:00:00Z', 'description': 'When a lead is created, make a post to reminder the owner of the lead', 'flow': '{"Metadata" : {"actionCalls" : [ {"actionName" : "chatterPost","actionType" : "chatterPost","description" : "make a post to owner of the lead","flowTransactionModel" : "CurrentTransaction","inputParameters" : [ {"name" : "text","value" : {"stringValue" : "Hey! You have a new Lead - {!$Record.LastName}. Remember to follow up!"}}, {"name" : "subjectNameOrId","value" : {"elementReference" : "$Record.OwnerId"}} ],"label" : "Post to chatter","locationX" : 176,"locationY" : 323,"name" : "Post_to_chatter","nameSegment" : "chatterPost","storeOutputAutomatically" : true,"versionSegment" : 1} ],"apiVersion" : 58.0,"description" : "make a post to owner of new lead"

TO delete flows

In [17]:
client.data_object.delete(
  "357f9348-3e38-442b-9770-633b8d0808f1",
  class_name="Flow",
  consistency_level=weaviate.data.replication.ConsistencyLevel.ALL,  # default QUORUM
)